In [26]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()
tf.set_random_seed(1)


In [27]:
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

lr=0.001
training_iters=100000
batch_size=128
n_inputs=28
n_steps=28
n_hidden_unists=128
n_classes=10

x=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y=tf.placeholder(tf.float32,[None,n_classes])

weights={
        'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_unists])),
        'out':tf.Variable(tf.random_normal([n_hidden_unists,n_classes]))
        }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_unists,])),
        'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))
        }


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
def RNN(X,weights,biases):
    X=tf.reshape(X,[-1,n_inputs])
    X_in=tf.matmul(X,weights['in'])+biases['in']
    X_in=tf.reshape(X_in,[-1,n_steps,n_hidden_unists])
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(n_hidden_unists,forget_bias=1.0,state_is_tuple=True)
    init_state=lstm_cell.zero_state(batch_size,dtype=tf.float32)
    outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=init_state,time_major=False)
    #results=tf.matmul(final_state[1],weights['out'])+biases['out']
    outputs=tf.unstack(tf.transpose(outputs,[1,0,2]))
    results=tf.matmul(outputs[-1],weights['out'])+biases['out']
    return results

In [29]:
pred=RNN(x,weights,biases)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
train_op=tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()

In [25]:
with tf.Session() as sess:
    sess.run(init)
    step=0
    while step*batch_size < training_iters:
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape([batch_size,n_steps,n_inputs])
        sess.run([train_op],feed_dict={x:batch_xs,y:batch_ys})
        if step % 20 ==0:
            print(sess.run(accuracy,feed_dict={x:batch_xs,y:batch_ys}))
        step+=1

InternalError: Blas GEMM launch failed : a.shape=(3584, 28), b.shape=(28, 128), m=3584, n=128, k=28
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape, Variable/read)]]

Caused by op 'MatMul', defined at:
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
   